When members have multiple email addresses on file, Broadstripes exports them all. By default, each email is in a different row. For many members, we even have multiple Dartmouth emails on file. Meanwhile, Mailchimp only allows one email per contact. It's a bit of a mess.

Some manual trial and error shows that Broadstripes seems to always list the member's "Primary" email first. So this script prunes the exported contact list to keep only the first Dart email for each member.

In [4]:
import pandas as pd
from pathlib import Path

infile = Path("data/Contact Info for Export.csv")
df = pd.read_csv(infile)

# Drop non-Dartmouth emails and aggregate by Broadstripes ID
emails = df["Email"]
df_dart = df[~emails.isnull() & emails.str.contains("dartmouth.edu", case=False)]
df_dart = df_dart.groupby("Broadstripes ID")

# Uncomment to see all the contacts that do have multiple Dart emails on file
# agg = df_dart.agg(func=list).reset_index()
# dups = agg.loc[[len(e) > 1 for e in agg["Email"]]]
# dups.to_csv("dups.csv", index=False)

# Keep only the first Dartmouth email for each ID
df_dart = df_dart.first().reset_index()
assert (df_dart["Broadstripes ID"].sort_values() == pd.Series(sorted(df["Broadstripes ID"].unique()))).all()

df_dart = df_dart[[
    "Nickname",
    "Last Name",
    "Phone",
    "Email"
]]

outfile = infile.parent / (infile.stem + " - Pruned for Mailchimp.csv")
df_dart.to_csv(outfile, index=False)